In [1]:
!pip install torch tiktoken transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:0000:01


In [2]:
import requests

url = "https://www.gutenberg.org/cache/epub/84/pg84.txt"
filename = "pg84.txt"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Successfully downloaded '{filename}'")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

Successfully downloaded 'pg84.txt'


In [3]:
with open("pg84.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom Dataset for the text data

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [5]:
import tiktoken

def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [6]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[171, 119, 123, 464]]), tensor([[ 119,  123,  464, 4935]])]


In [7]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 119,  123,  464, 4935]]), tensor([[  123,   464,  4935, 20336]])]


In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  171,   119,   123,   464],
        [ 4935, 20336, 46566,   286],
        [45738,    26,  1471,    11],
        [  383, 12495, 42696,   198],
        [  220,   220,   220,   220],
        [  198,  1212, 47179,   318],
        [  329,   262,   779,   286],
        [ 2687,  6609,   287,   262]])

Targets:
 tensor([[  119,   123,   464,  4935],
        [20336, 46566,   286, 45738],
        [   26,  1471,    11,   383],
        [12495, 42696,   198,   220],
        [  220,   220,   220,   198],
        [ 1212, 47179,   318,   329],
        [  262,   779,   286,  2687],
        [ 6609,   287,   262,  1578]])


In [8]:
import tiktoken

# Get the gpt2 tokenizer
tokenizer_bpe = tiktoken.get_encoding("gpt2")

# Encode the raw_text
tokens_bpe = tokenizer_bpe.encode(raw_text)

# Print the number of tokens
print(f"Total number of tokens in raw_text using GPT-2 BPE encoding: {len(tokens_bpe)}")

Total number of tokens in raw_text using GPT-2 BPE encoding: 106361


In [9]:
vocab_size = tokenizer_bpe.n_vocab
print(vocab_size)
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

50257


In [10]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[  171,   119,   123,   464],
        [ 4935, 20336, 46566,   286],
        [45738,    26,  1471,    11],
        [  383, 12495, 42696,   198],
        [  220,   220,   220,   220],
        [  198,  1212, 47179,   318],
        [  329,   262,   779,   286],
        [ 2687,  6609,   287,   262]])

Inputs shape:
 torch.Size([8, 4])


In [11]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [12]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [13]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
